In [15]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import tensorflow as tf
import os
from nltk.stem import WordNetLemmatizer

# Load pre-trained TensorFlow model
saved_model_path = '/objectivity_vs_subjectivity/subjectivity_classifier/'
subjectivity_estimator = tf.saved_model.load(saved_model_path)

# Speculative cues taken from 
speculative_cues = ['may','might','can','would','should','could',
                    'think','suggest','question','presume',
                    'suspect','indicate','suppose','seem','appear','expect',
                    'probable','likely','possible','apparently','unsure',
                    'if','or','and/or','either','versus','vs']

# Modal verbs taken from Cambridge Dictionary (2022) 'Modal verbs and modality'
modal_verb_list = ['can','could','may','might','will','shall','would','should','must',
                   'dare','need','ought','used','going','able']

# Subjective adjectives taken from Wiebe (2000) 'Learning subjective adjectives from corpora'
subjective_adjective_list = pd.read_csv('https://people.cs.pitt.edu/~wiebe/pubs/aaai00/adjsMPQA', header=None)[0].to_list()
subjective_adjective_list = [word.strip() for word in subjective_adjective_list]

def measure_subjective_sentence_freq(sent_list):
    subjective_sent_count = None
    return subjective_sent_count/len(sent_list)

def measure_speculative_sentence_freq(sent_list, speculative_cues=speculative_cues):
    wnl = WordNetLemmatizer()
    lemmatised_sent_list = [[wnl.lemmatize(word) for word in sent] for sent in sent_list]
    speculative_sent_count = sum([1 for lemma_sent in lemmatised_sent_list if any([lemma in speculative_cues for lemma in lemma_sent])])
    return speculative_sent_count/len(sent_list)

def measure_modal_verb_freq(text_list, modal_verb_list=modal_verb_list):
    modal_verb_count = sum([word in modal_verb_list for word in text_list])
    return modal_verb_count/len(text_list)

def measure_subjective_adjective_freq(text_list, subjective_adjective_list=subjective_adjective_list):
    subjective_adjective_count = sum([word in subjective_adjective_list for word in text_list])
    return subjective_adjective_count/len(text_list)

In [19]:
high_freq = measure_speculative_sentence_freq([['this','sentence','can','have','more','speculative','cues'],
                                               ['however','i','think','that','this','sentence','has','enough']])
low_freq = measure_speculative_sentence_freq([['this','sentence','has','no','speculative','cues'],
                                              ['this','sentence','has','none','also']])
print("High speculation frequency: {:.1%}".format(high_freq))
print("Low speculation frequency: {:.1%}".format(low_freq))

High speculation frequency: 100.0%
Low speculation frequency: 0.0%


In [20]:
high_freq = measure_modal_verb_freq(['this','sentence','would','have','ought','to','need','more','modal','verbs'])
low_freq = measure_modal_verb_freq(['this','sentence','has','not','got','any','modal','verbs'])
print("High modal frequency: {:.1%}".format(high_freq))
print("Low modal frequency: {:.1%}".format(low_freq))

High modal frequency: 30.0%
Low modal frequency: 0.0%


In [21]:
high_freq = measure_subjective_adjective_freq(['this','sentence','has','an','extraordinary','number','of','subjective','adjectives'])
low_freq = measure_subjective_adjective_freq(['this','sentence','has','not','got','any','subjective','adjectives'])
print("High subj. adj. frequency: {:.1%}".format(high_freq))
print("Low subj. adj. frequency: {:.1%}".format(low_freq))

High subj. adj. frequency: 11.1%
Low subj. adj. frequency: 0.0%
